# Cálculo de áreas de figuras geométricas usando redes neuronales en TensorFlow
Este notebook muestra cómo una red neuronal puede aprender a calcular áreas de diferentes figuras (cuadrados, rectángulos, triángulos, círculos) a partir de sus parámetros.

Autor: Dario Coronel  
Contacto: dario.coronel@gmail.com

In [ ]:
# Instalación (si es necesario, descomenta la siguiente línea)
# !pip install tensorflow matplotlib scikit-learn pandas
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Preparación del dataset
Codificamos el tipo de figura con one-hot y sus parámetros. Ejemplo: 
- Cuadrado: [1,0,0,0, lado, 0, 0, 0]
- Rectángulo: [0,1,0,0, 0, base, altura, 0]
- Triángulo: [0,0,1,0, 0, base, altura, 0]
- Círculo: [0,0,0,1, 0, 0, 0, radio]

La salida es el área esperada.

In [ ]:
# Dataset de ejemplo
import numpy as np
X = np.array([
    [1,0,0,0, 4, 0, 0, 0],   # Cuadrado de lado 4 (área 16)
    [1,0,0,0, 10,0, 0, 0],   # Cuadrado de lado 10 (área 100)
    [0,1,0,0, 0, 3, 5, 0],   # Rectángulo base 3, altura 5 (área 15)
    [0,1,0,0, 0, 7, 2, 0],   # Rectángulo base 7, altura 2 (área 14)
    [0,0,1,0, 0, 8, 2, 0],   # Triángulo base 8, altura 2 (área 8)
    [0,0,1,0, 0, 6, 9, 0],   # Triángulo base 6, altura 9 (área 27)
    [0,0,0,1, 0, 0, 0, 7],   # Círculo radio 7 (área 153.938)
    [0,0,0,1, 0, 0, 0, 3],   # Círculo radio 3 (área 28.274)
    [1,0,0,0, 2, 0, 0, 0],   # Cuadrado de lado 2 (área 4)
    [0,1,0,0, 0, 10,1, 0],   # Rectángulo base 10, altura 1 (área 10)
    [0,0,1,0, 0, 3, 5, 0],   # Triángulo base 3, altura 5 (área 7.5)
    [0,0,0,1, 0, 0, 0, 5],   # Círculo radio 5 (área 78.5398)
], dtype=float)

y = np.array([
    16, 100, 15, 14, 8, 27, 
    np.pi*7*7, np.pi*3*3, 4, 10, 7.5, np.pi*5*5
], dtype=float)

print('Forma de X:', X.shape)
print('Primeros X:', X[:3])
print('Áreas esperadas:', y[:3])

## Definición y entrenamiento del modelo (con EarlyStopping)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

early = EarlyStopping(monitor='loss', patience=30, restore_best_weights=True)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=16, activation='relu', input_shape=[8]),
    tf.keras.layers.Dense(units=8, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.05),
    loss='mean_squared_error'
)

print('Entrenando modelo...')
history = model.fit(X, y, epochs=1000, verbose=False, callbacks=[early])
print('Modelo entrenado.')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Entrenamiento')
plt.show()

## Pruebas de predicción individuales

In [ ]:
import numpy as np
# Cuadrado lado 6
test = np.array([[1,0,0,0, 6,0,0,0]])
pred = model.predict(test)[0][0]
print('Área cuadrado lado 6, predicha:', pred, 'esperada:', 36)

# Rectángulo base 5, altura 3
test = np.array([[0,1,0,0, 0,5,3,0]])
pred = model.predict(test)[0][0]
print('Área rectángulo 5x3, predicha:', pred, 'esperada:', 15)

# Triángulo base 8, altura 6
test = np.array([[0,0,1,0, 0,8,6,0]])
pred = model.predict(test)[0][0]
print('Área triángulo 8x6, predicha:', pred, 'esperada:', 24)

# Círculo radio 4
test = np.array([[0,0,0,1, 0,0,0,4]])
pred = model.predict(test)[0][0]
print('Área círculo radio 4, predicha:', pred, 'esperada:', np.pi*4*4)

## Evaluación con ejemplos aleatorios y visualización gráfica

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import random
import pandas as pd
import matplotlib.pyplot as plt

def generar_ejemplo():
    tipo = random.choice(['cuadrado', 'rectangulo', 'triangulo', 'circulo'])
    if tipo == 'cuadrado':
        lado = random.uniform(1, 15)
        x = [1,0,0,0, lado,0,0,0]
        area = lado * lado
        etiqueta = f'Cuadrado ({lado:.1f})'
    elif tipo == 'rectangulo':
        base = random.uniform(1, 15)
        altura = random.uniform(1, 15)
        x = [0,1,0,0, 0,base,altura,0]
        area = base * altura
        etiqueta = f'Rectángulo ({base:.1f}x{altura:.1f})'
    elif tipo == 'triangulo':
        base = random.uniform(1, 15)
        altura = random.uniform(1, 15)
        x = [0,0,1,0, 0,base,altura,0]
        area = (base * altura) / 2
        etiqueta = f'Triángulo ({base:.1f}x{altura:.1f})'
    else:
        radio = random.uniform(1, 15)
        x = [0,0,0,1, 0,0,0,radio]
        area = np.pi * radio * radio
        etiqueta = f'Círculo (r={radio:.1f})'
    return np.array(x), area, etiqueta

ejemplos_X = []
ejemplos_y = []
etiquetas = []

for _ in range(20):
    x, area, etiqueta = generar_ejemplo()
    ejemplos_X.append(x)
    ejemplos_y.append(area)
    etiquetas.append(etiqueta)

ejemplos_X = np.array(ejemplos_X)
ejemplos_y = np.array(ejemplos_y)
predicciones = model.predict(ejemplos_X).flatten()

# Métricas de error
mae = mean_absolute_error(ejemplos_y, predicciones)
mse = mean_squared_error(ejemplos_y, predicciones)
print(f"MAE: {mae:.3f}, MSE: {mse:.3f}")

# Gráfico real vs predicción
plt.figure(figsize=(10,5))
plt.scatter(ejemplos_y, predicciones, color="royalblue")
plt.plot([0, max(ejemplos_y)], [0, max(ejemplos_y)], 'r--', label='Predicción perfecta')
plt.xlabel("Área real")
plt.ylabel("Área predicha")
plt.title("Comparación de áreas reales vs predichas (ejemplos aleatorios)")
plt.legend()
plt.grid(True)
plt.show()

# Tabla resumen
tabla = pd.DataFrame({
    'Etiqueta': etiquetas,
    'Área real': ejemplos_y,
    'Predicción': predicciones,
    'Error abs': np.abs(ejemplos_y - predicciones)
})
tabla.sort_values('Error abs')